In [58]:
import requests
import pandas as pd
from flatten_json import flatten
from utils.io import dict_to_yaml
from utils.io import yaml_to_dict
import numpy as np
from typing import List, Tuple, Dict, Union
from itertools import compress
import pandas as pd
import re
import datetime as dt
from cleanco import basename
from json import dumps
import matplotlib.pyplot as plt
import seaborn as sns
import postproc
import matplotlib.colors as mc

id = "lgulden"
apikey = "4d1a4bc3be920e859b3862a25d3725d741028d42"
data_gov_api_key = "n4TEYUedn3STcnsxQi7JllzXZgfB6Dqg5pV2068E"
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [ ]:
config = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/config_ccs_lda.yml"
)
term_list_dict = yaml_to_dict(config["search_term_list_path"])
law_list_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/ccs_laws.yml"
)

In [ ]:
authenticated_session = api_authenticate(
    config["authentication_endpoint"],
    config["lda_username"],
    config["lda_apikey"],
)
result = authenticated_session.get(
    config["filings_endpoint"],
    params={"filing_specific_lobbying_issues": f"{search_string}"},
)

In [59]:
def get_list_govt_entities(entity_endpoint: str, session: object):
    """Queries constants endpoint to get a standardized list of government entities"""
    govt_entities = session.get(entity_endpoint, timeout=60)
    entity_df = pd.DataFrame(govt_entities.json())
    entities = sorted([x.lower() for x in list(entity_df["name"])])
    return entities


def api_authenticate(
    authentication_endpoint,
    username,
    apikey,
    username_key="username",
    password_key="password",
):
    """logs in to the api using provided authentication endpoint and credentials"""
    authenticated_session = requests.Session()

    authenticated_session.post(
        authentication_endpoint,
        json={
            username_key: username,
            password_key: apikey,
        },
        timeout=60,
    )

    return authenticated_session


def terms_present(phrase, term_list, find_any=True):
    """utility function to see if terms in terms_list are present in a given phrase
    Args:
        phrase: phrase to be searched
        term_list: list of strings to be searched for within phrase
        find_any: boolean -- true if function should return true if any of the terms are in the phrase;
            false if the function should only return true if all terms are present
    Returns:
        int: 1 if one or more of the terms are present in phrase, 0 otherwise
    """
    if not isinstance(phrase, str):
        raise TypeError("phrase must be a string")

    if phrase is None:
        return 0
    n_present = 0
    for term in term_list:
        if not isinstance(term, str):
            raise TypeError(" all terms in term_list must be strings")
        if term.lower() in phrase.lower():
            if find_any:
                return 1
            n_present += 1
    if n_present == len(term_list):
        return 1
    return 0


def substitute(
    x: str,
    use_basename: bool = False,
    re_types: str = r"[^\w\s]",
    replace_str: str = "",
):
    """wrapper function for regular expression substitute funciton, linked with basename lib"""
    # use basename for company names
    if not isinstance(x, str):
        x = ""
    if use_basename:
        return basename(re.sub(re_types, replace_str, x))
    # don't use basename for general strings
    return re.sub(re_types, replace_str, x).rstrip().lstrip()


def parse_client_names(
    input_df, config, client_rename_col="client_rename"
) -> Tuple[pd.DataFrame, Dict[str, str]]:
    """bespoke function for parsing organization names from LDA queries
    Args:
        input_df: input pandas dataframe (read raw from files saved after api access)
        config: configuration dictionary containing details on how to handle names
        client_rename_column: name for new column in dataframe that is returned
    Returns:
        output_df: processed column with renamed clients and removed organziations
        client_name_rename_dict: dictionary used for renaming (contains original and new names)
    """
    output_df = input_df.copy(deep=True)

    starting_client_names = sorted(list(output_df.client_name.unique()))
    client_names = sorted(list(output_df.client_name.unique()))
    client_names = [
        substitute(company_name, use_basename=True) for company_name in client_names
    ]

    # take rightmost component of client name
    for term in config["take_terms_to_the_right_of_these_words"]:
        client_names = [x.split(term)[-1] for x in client_names]
    # take leftmost component of client name
    for term in config["take_terms_to_the_left_of_these_words"]:
        client_names = [x.split(term)[0] for x in client_names]
    # remove words
    for n in config["remove_these_phrases"]:
        client_names = [x.replace(n, "") for x in client_names]
    # get rid of double spaces
    client_names = [x.replace("  ", " ") for x in client_names]
    # trim spaces on ends of names
    client_names = [x.rstrip().lstrip() for x in client_names]

    # extract shorter, well-known names from longer names
    for co in config["use_these_name_subsets_for_organiztions"]:
        client_names = [co if co in x else x for x in client_names]

    # bespoke replacements and handling of mergers
    for key, value in config["replace_names_on_left_with_names_on_right"].items():
        client_names = [x.replace(key, value) for x in client_names]

    # make a renaming dictionary
    client_name_rename_dict = dict(zip(starting_client_names, client_names))

    # add the 'remove' companies to the rename dictionary
    remove_companies = config["remove_companies_containing_these_terms"]
    for x in client_name_rename_dict.keys():
        if terms_present(client_name_rename_dict[x], remove_companies, find_any=True):
            client_name_rename_dict[x] = "remove"

    # make new column with renames
    output_df[client_rename_col] = [
        client_name_rename_dict[x] for x in output_df.client_name
    ]
    output_df = output_df.loc[output_df[client_rename_col] != "remove"]

    return output_df, client_name_rename_dict


def get_smarties(
    row: Union[pd.Series, List[Union[bool, int]]], names: List[str]
) -> List[str]:
    if isinstance(row, pd.Series):
        return list(compress(names, row[names].values.tolist()))
    if isinstance(row, list):
        return list(compress(names, row))
    raise TypeError("get_smarties argument 'row' must be a Pandas Series or a list")


def get_latest_filings(
    df: pd.DataFrame, groupby_cols: List[str], date_col="filing_dt_posted"
):
    """get only the latest filing for a given lobbying firm, client, and quarter"""
    if df[date_col].dtype == str:
        df[date_col] = [dt.datetime.fromisoformat(d) for d in df[date_col]]

    df.sort_values(by=date_col, ascending=False, inplace=True)

    df = df.groupby(groupby_cols).first().reset_index()
    return df


def invert_sector_dict(sectors_path) -> Dict[str, str]:
    """reads in the sector assignment yaml to dict; inverts dict s.t. each company is a key"""
    sector_assignments = yaml_to_dict(sectors_path)

    all_companies = []
    for _, value in sector_assignments.items():
        all_companies = all_companies + value
    # print(all_companies)

    company_sector_dict = {}
    for k, vv in sector_assignments.items():
        for v in vv:
            company_sector_dict = company_sector_dict | {v: k}

    return company_sector_dict

In [ ]:
config_info = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/config_ccs_lda.yml"
)
groupby_cols = [
    "filing_year",
    "filing_period",
    "client_id",
    "registrant_id",
    "activity_id",
]
entities = get_list_govt_entities(
    config_info["entity_endpoint"],
    session=api_authenticate(
        config_info["authentication_endpoint"],
        config_info["lda_username"],
        config_info["lda_apikey"],
    ),
)
remove_sector_descriptions = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sector_company_description_terms.yml"
)["remove"]

df_list = []
rename_dict = {}
for i in range(1, 19):  # 159):
    api_results_df = pd.read_csv(
        # f"/Volumes/Samsung_T5/data/lobbying/ccslaws/ccs_lda_filings_{i}.csv",
        f"/Volumes/Samsung_T5/data/lobbying/ccs/ccs_lda_filings_{i}.csv",
        index_col=[0],
        # parse_dates=["filing_dt_posted"],
        dtype={"filing_year": int},
        low_memory=False,
    )
    # remove unwanted filing types
    api_results_df = api_results_df.loc[
        [x[0] != "R" for x in api_results_df.filing_type]
    ]

    # parse company names, remove unwanted names, add to list
    df, this_rename_dict = parse_client_names(
        api_results_df, yaml_to_dict(config_info["organization_name_handling_path"])
    )

    # append the rename dictionary to the whole thing
    rename_dict = rename_dict | this_rename_dict
    # compress entities into a single string column and get rid of entity columns
    df["entities"] = df[entities].T.apply(lambda x: dumps(get_smarties(x, entities)))
    df.drop(entities, axis=1, inplace=True)
    df["clean_description"] = [
        substitute(d, use_basename=True) for d in df["description"]
    ]
    df["clean_client_general_description"] = [
        substitute(d, use_basename=False) for d in df["client_general_description"]
    ]
    df["client_rename"] = [
        "remove" if terms_present(x, remove_sector_descriptions) else n
        for x, n in zip(df.clean_client_general_description, df.client_rename)
    ]
    df["client_rename"] = [
        "remove" if terms_present(x, remove_sector_descriptions) else x
        for x in df.client_rename
    ]

    df = get_latest_filings(df, groupby_cols)
    df = df.loc[df.client_rename != "remove"]
    df_list.append(df)

ccs_df = pd.concat(df_list)
ccs_df = get_latest_filings(ccs_df, groupby_cols)
# fill in nans/nones with empty string for description and rename of client
ccs_df.clean_client_general_description = (
    ccs_df.clean_client_general_description.fillna("")
)
ccs_df.client_rename = ccs_df.client_rename.fillna("")
ccs_df["batch"] = "ccs description and/or ccs specific laws and bills"
ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccs_compiled.csv")
# ccs_df["batch"] = "relevant laws"
# ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccslaws_compiled.csv")

In [ ]:
# join different query results into a single dataframe
tmp_df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccs_compiled.csv")
law_df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccslaws_compiled.csv")
ccs_df = pd.concat([tmp_df, law_df])
ccs_df.sort_values(by=["batch"], ascending=True, inplace=True)
ccs_df.drop_duplicates(
    subset=["filing_uuid", "activity_id"], keep="first", inplace=True
)
ccs_df.reset_index(inplace=True)
ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccsall_compiled.csv")

In [ ]:
# write out updated, merged, slightly processed ccsall
ccs_df.clean_client_general_description = (
    ccs_df.clean_client_general_description.fillna("")
)
ccs_df.client_rename = ccs_df.client_rename.fillna("")
ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccsall_compiled_revised.csv")

In [70]:
ccs_df = pd.read_csv(
    #    "/Volumes/Samsung_T5/data/lobbying/ccs/ccsall_compiled.csv", index_col=[0]
    "/Volumes/Samsung_T5/data/lobbying/ccs/ccsall_compiled_revised.csv",
    index_col=[0],
)
ccs_df.clean_client_general_description = (
    ccs_df.clean_client_general_description.fillna("")
)
ccs_df.client_rename = ccs_df.client_rename.fillna("")

In [134]:
description_dict = yaml_to_dict("sector_descriptions.yml")
sector_list = sorted(list(description_dict.keys()))

In [287]:
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
companies = [c.rstrip().lstrip() for c in list(inverted_sector_dict.keys())]

In [288]:
ccs_df.loc[
    ~ccs_df.client_rename.isin(companies)
    # & (ccs_df.clean_client_general_description == "")
].client_rename.nunique()

1260

In [266]:
for c, n in (
    ccs_df.loc[
        ~ccs_df.client_rename.isin(companies)
        # & (ccs_df.clean_client_general_description == "")
    ]
    .clean_client_general_description.value_counts()
    .items()
):
    # if ("geo" in c.lower()) | ("college" in c.lower()):
    print(f"- {c}")

- 
- Manufacturing
- Advocacy
- Reclamation Systems
- Government Affairs Consulting
- Global driver risk management company
- provider of enterprise cloud technology solutions
- 226Data centers
- 226Electric Vehicle Charging Stations
- Manufacturer of gates and software to improve irrigation canal efficiency and safety
- 226An association formed to help shape the national publicprivate partnership marketplace
- American aerospace manufacturer and space transportation services company
- Motor manufacturing
- Aviation company
- Enterprise software company
- develop and advocate for the adoption of model building codes
- Cybersecurity company
- Manufacturer of barcode and location systems technology
- UKbased energy company
- Township
- Government affairs
- Technology
- Government relations
- Port Operations and Stevedoring Services
- Helicopter services company
- Building systems manufacturer
- National trade association comprised of manufacturers
- Materials manufacturing
- PMI is the i

In [261]:
description_dict = yaml_to_dict("sector_descriptions.yml")
tmp = sorted(
    [
        f"'{x.lower()}'" if (x[0] == " ") | (x[-1] == " ") else f"{x.lower()}"
        for x in list(set(description_dict["remove these organizations"]["keep"]))
    ]
)
for t in tmp:
    print(f" - {t}")

 - ' aging'
 - ' care '
 - ' cure '
 - ' drone '
 - ' eye'
 - ' fish '
 - ' gaming '
 - ' heart '
 - ' housing '
 - ' la raza '
 - ' lending'
 - ' liver '
 - ' marrow'
 - ' pr '
 - 60 plus association
 - aarp
 - abbott laboratories
 - abbvie
 - academy advisors
 - access
 - accountants
 - acte
 - actuaries
 - acuity brands lighting
 - acuityhealtcare
 - adage
 - adobe
 - adolescent
 - advamed
 - advanced micro devices
 - advantage pressurepro
 - advertising
 - aetna
 - affiliated computer state and local solutions
 - afford
 - aflac
 - aft
 - afterschool alliance
 - agilent
 - aids
 - alcatellucent
 - alcon laboratories
 - alkermes
 - allergan
 - alliance for biosecurity
 - allied fiber
 - ally financial
 - alticor
 - altria
 - alzheimers
 - alzheimers
 - amazon
 - american ambulance association
 - american association for justice
 - american association of classified school employees
 - american bakers association
 - american benefits council
 - american civil liberties union
 - ameri

In [339]:
# FIND SPECIFIC INDUSTRIES
which_sector = "oil and gas"  # "auto and transportation"  # "renewable energy"  # "chemicals"  # "cement"  # "engineering, technology, and consulting"  # "nuclear"  # "remove these organizations"
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
companies = [c.rstrip().lstrip() for c in list(inverted_sector_dict.keys())]
description_dict = yaml_to_dict("sector_descriptions.yml")
sector_list = sorted(list(description_dict.keys()))
org_names = []
companies_list = companies  # list(inverted_sector_dict.keys())
for n, row in ccs_df.iterrows():
    phrase = row["clean_client_general_description"].lower()
    client_rename = row["client_rename"].lower()
    if row["client_rename"] in companies:
        pass
    elif terms_present(
        phrase, description_dict[which_sector]["exclude"], find_any=True
    ):
        pass
    elif terms_present(
        client_rename,
        description_dict[which_sector]["exclude"],
        find_any=True,
    ):
        pass
    elif terms_present(phrase, description_dict[which_sector]["keep"], find_any=True):
        org_names.append((row["client_rename"]))
    elif terms_present(
        client_rename, description_dict[which_sector]["keep"], find_any=True
    ):
        org_names.append((row["client_rename"]))
org_list = sorted(list(set(org_names)))
for i in org_list:
    if i.rstrip().lstrip() in companies:
        pass
    else:
        # print(f"- {i}")
        print(f"'{i}',")

'EXPLORATION',
'NEW ENERGY TECHNOLOGY MANUFACTURERS ALLIANCE',
'RECYCLED ENERGY DEVELOPMENT',
'UNITY GLOBAL ENERGY',
'VIEW',


In [422]:
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
companies = [c.rstrip().lstrip() for c in list(inverted_sector_dict.keys())]
for c in sorted(list(set(ccs_df.client_rename.unique()) - set(companies))):
    print(f"- {c}")
print(
    f"************** {len(sorted(list(set(ccs_df.client_rename.unique()) - set(companies))))}"
)

- 
- ABIR
- ADAPDIX
- ADVANCED DRAINAGE SYSTEMS
- AEROSEAL
- AIM
- AIRCONDITIONING HEATING AND REFRIGERATION INSTITUTE
- AKER BIOMARINE
- AKUTAN
- ALEN
- ALEUT
- ALLIANCE FOR INFRASTRUCTURE REUSE AND REDEVELOPMENT
- ALLISON TRANSMISSION
- ALTAFIBER AND ITS SUBSIDIARIES
- ALUTIIQ MANAGEMENT SERVICES
- AM GENERAL
- AMAC ACTION
- AMERICAN AVK
- AMERICAN INSTITUTE OF PHYSICS
- AMERICAN NITRILE
- AMERICAN PRESIDENT LINES
- AMERICAN PUBLIC TRANSPORTATION ASSOCIATION APTA
- AMERICAN PUBLIC WORKS ASSOCIATION
- AMERICAN RECOVERY ASSOCIATION
- AMERICAN ROAD AND TRANSPORTATION BUILDERS ASSOCIATION
- AMERICAN SMALL MANUFACTURING COALITION
- AMERICAN SUPPLY ASSOCIATION
- AMERICAN TRAFFIC SAFETY ASSOCIATION
- AMERICAS SERVICE COMMISSIONS
- AMSTY
- ANALYTICAL GRAPHICS
- ANDURIL INDUSTRIES
- ANSYS
- ANTIDEFAMATION LEAGUE
- APOLLO MANAGEMENT HOLDING
- APOTEX
- ARCADIACA
- ARENA OFFSHORE
- AREVA
- ARGO AI
- ARM
- AROTECH
- ASCENSION
- ASHRAE
- ASSOCIATION FOR COMMUTER TRANSPORTATION
- ASSOCIATION FOR IM

In [410]:
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
companies = [c.rstrip().lstrip() for c in list(inverted_sector_dict.keys())]
not_assigned = sorted(list(set(ccs_df.client_rename.unique()) - set(companies)))

list_of_companies = org_list
list_of_companies = not_assigned
# list_of_companies = sorted(list(set(sectors_dict[which_sector])))
for org in list_of_companies:
    descriptions = list(
        ccs_df.loc[
            ccs_df.client_rename == org
        ].clean_client_general_description.unique()
    )
    term = "flight"
    if (term in org.lower()) | any([term in x for x in descriptions]):
        print(f"- {org}")
        print(descriptions)

- SIGNATURE FLIGHT SUPPORT
['Support for Business and Private Aviation', 'Aviation fixed based operator']
- WORLDWIDE FLIGHT
['Private equity and alternative investment management']


In [403]:
len(not_assigned)

927

In [300]:
# read it back in and print
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
for i in sorted(list(set(sectors_dict[which_sector]))):
    print(f"- {i}")

- 3M
- AIR LIQUIDE
- AIR PRODUCTS AND CHEMICALS
- AIRCONDITIONING HEATING REFRIGERATION INSTITUTE
- AKJ INDUSTRIES
- ALBEMARLE
- ALKALINE WATER COMPANY
- ALLIANCE FOR RESPONSIBLE ATMOSPHERIC POLICY
- ALLKEM
- ALTERNATIVE FUELS CHEMICALS COALITION
- AMERICAN CHEMICAL SOCIETY
- AMERICAN CHEMISTRY COUNCIL
- AMERICAN FUEL PETROCHEMICAL MANUFACTURERS
- AMERICAN FUEL PETROCHEMICAL MANUFACTURERS AFPM
- AMERICAN PACIFIC
- AMERICAS STYRENICS
- ARCHROMA US
- ARIZONA CHEMICAL
- ARKEMA
- ASHLAND
- BASF
- BAYER
- BOOSTER FUELS
- BRASKEM AMERICA
- CARBON SINK
- CELANESE
- CF INDUSTRIES
- CHART INDUSTRIES
- CHEMMOD
- CHEMOURS
- CHZ TECHNOLOGIES
- CLEAN FUELS ALLIANCE AMERICA
- CODEXIS
- CORN REFINERS ASSOCIATION
- COUNTY OF PLACER
- COVESTRO
- DAIKIN AMERICA
- DANIMER SCIENTIFIC
- DG FUELS
- DIAMOND PLASTIC
- DOW
- DUPONT
- EASTMAN CHEMICAL
- ECOLAB
- EMERGENT BIOSOLUTIONS
- EVONIK
- FERTILIZER INSTITUTE
- FIRSTELEMENT FUEL
- FMC
- GENERAL SYNFUELS INTERNATIONAL GSI
- GREEN CHEMISTRY COMMERCE COUNCIL

In [ ]:
ar = ccs_df.loc[
    [
        (
            x
            == "MRGCD delivers irrigation water to Pueblos farmers and maintains river habitat"
        )
        and (n in org_list)
        for x, n in zip(ccs_df.clean_client_general_description, ccs_df.client_rename)
    ]
].client_rename.unique()
ar

In [ ]:
ccs_df.loc[ccs_df.client_rename == ar[0]].clean_client_general_description.unique()

In [160]:
set(sectors_dict["auto and transportation"]).intersection(
    set(sectors_dict["remove these organizations"])
)

set()

In [417]:
for co in ["KOPPERS", "KOPPERS INDUSTRIES"]:
    for i in list(
        ccs_df.loc[
            ccs_df.client_rename
            == co
            # ].clean_client_general_description.unique()
        ].client_name.unique()
    ):
        if "ELEPHANT" in i:
            print(f"'{i}':'DOW',")
        else:
            print(f"'{i}':'{substitute(i,use_basename=True)}',")

'KOPPERS, INC.':'KOPPERS',
'KOPPERS INDUSTRIES':'KOPPERS INDUSTRIES',


In [415]:
portland_replace_dict = {
    'KOPPERS, INC.':'KOPPERS',
    'KOPPERS INDUSTRIES':'KOPPERS',
    "NATIONAL PROPANE GAS ASSOCIATION (NPGA)": "NATIONAL PROPANE GAS ASSOCIATION",
    "NEWFIELD EXPLORATION COMPANY": "ENCANA",
    "GREENHOUSE GAS SERVICES, LLC": "GREENHOUSE GAS SERVICES",
    "MRS. MARY F. PEARSALL": "ESTATE OF JANE CARBIS",
    "MARTIN G HAMBERGER AND ASSOCIATES (AMERICAN PUBLIC TRANSPORTATION ASSOCIATION)": "AMERICAN PUBLIC TRANSPORTATION ASSOCIATION",
    "ARRAY TECH, INC.": "ARRAY TECHNOLOGIES",
    "COVANTA ENERGY CORPORATION": "REWORLD",
    "REWORLD (FORMERLY COVANTA)": "REWORLD",
    "COVANTA ENERGY": "REWORLD",
    "AMERICAN EXPLORATION AND PRODUCTION COUNCIL (AXPC)": "AMERICAN EXPLORATION PRODUCTION COUNCIL",
    "AMERICAN EXPLORATION & PRODUCTION COUNCIL (AXPC)": "AMERICAN EXPLORATION PRODUCTION COUNCIL",
    "AES US SERVICES, LLC": "AES",
    "AES WR LIMITED PARTNERSHIP": "AES",
    "THE AES CORPORATION": "AES",
    "THE COMMERCIAL ENERGY WORKING GROUP": "COMMERCIAL ENERGY WORKING GROUP",
    "CALIFORNIA CREDIT UNION LEAGUE": "CALIFORNIA AND NEVADA CREDIT UNION LEAGUE",
    "THE COCA COLA COMPANY": "COCACOLA",
    "CALIFORNIA AND NEVADA CREDIT UNION LEAGUES": "CALIFORNIA AND NEVADA CREDIT UNION LEAGUE",
    "BLACKROCK FUNDS SERVICES GROUP LLC": "BLACKROCK",
    "BLACKROCK FUNDS SERVICES GROUP LLC FKA BLACKROCK, INC.": "BLACKROCK",
    "VOLTA INDUSTRIES, INC.": "VOLTA INDUSTRIES -- AS OF 2023 OWNED BY SHELL CORPORATION",
    "ACCENTURE FEDERAL SERVICES LLC": "ACCENTURE",
    "MCDERMOTT INTERNATIONAL INC F/K/A CHICAGO BRIDGE & IRON (CB&I)": "MCDERMOTT",
    "PIKE ASSOCIATES, LLC OBO WOODS HOLE MARTHA'S VINEYARD NANTUCKET STEAMSHIP AUTH.": "MARTHAS VINEYARD AND NANTUCKET STEAMSHIP AUTHORITY",
    "ON BEHALF OF WATER FOUNDATION, MEMBER OF WATER EQUITY CLIMATE RESILIENCE CAUCUS": "WATER EQUITY CLIMATE RESILIENCE CAUCUS",
    "INTERNATIONAL ASSOCIATION OF SHEET METAL, AIR, RAIL, AND TRANSPORTATION WORKERS": "INTERNATIONAL ASSOCIATION OF SHEET METAL AIR RAIL TRANSPORTATION WORKERS",
    "NORFOLK SOUTHERN RAILWAY CORPORATION": "NORFOLK SOUTHERN",
    "UNITED BROTHERHOOD OF CARPENTERS AND JOINERS OF AMERICA": "UNITED BROTHERHOOD OF CARPENTERS",
    "LABORERS INTERNATIONAL UNION OF NORTH AMERICA": "LABORERS INTERNATIONAL UNION OF NORTH AMERICA",
    "AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES, COUNCIL 118": "AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES",
    "INTERNATIONAL BROTHERHOOD OF BOILERMAKERS IRON SHIPBUILDERS BLACKSMITHS FORGE": "INTERNATIONAL BROTHERHOOD OF BOILERMAKERS",
    "SEAFARERS INTERNATIONAL UNION OF N.A. - AGLIW": "SEAFARERS INTERNATIONAL UNION",
    "NATIONAL CONFERENCE OF FIREMEN & OILERS DISTRICT OF LOCAL 32BJ/SEIU": "SERVICE EMPLOYEES INTERNATIONAL UNION",
    "TRANS UNION LLC": "TRANSUNION",
    "CAMP DRESSER & MCKEE": "CDM SMITH",
    "CHICAGO BRIDGE & IRON COMPANY N V": "MCDERMOTT",
    "DRESSER RAND COMPANY": "SIEMENS",
    "HDR ENGINEERING, INC.": "HDR",
    "COALITION FOR NORTH AMERICAN TRADE AND INVESTMENT": "COALITION FOR NORTH AMERICAN TRADE INVESTMENT",
    "AECOM TECHNOLOGY CORPORATION": "AECOM",
    "AECOM TECHNICAL SERVICES, INC.": "AECOM",
    "U.S. CHAMBER OF COMMERCE INSTITUTE FOR LEGAL REFORM": "US CHAMBER OF COMMERCE",
    "U.S. CHAMBER OF COMMERCE - C_TEC AND CCMC": "US CHAMBER OF COMMERCE",
    "NSTAR": "EVERSOURCE",
    "RIO TINTO AMERICA HOLDINGS, INC.": "RIO TINTO",
    "ENERGY TRANSFER PARTNERS": "ENERGY TRANSFER",
    "ENERGY TRANSFER EQUITY LP": "ENERGY TRANSFER",
    "THE INSTITUTE FOR LEGAL REFORM - U.S. CHAMBER OF COMMERCE": "US CHAMBER OF COMMERCE",
    "MASTERCARD INTERNATIONAL INCORPORATED": "MASTERCARD",
    "HUNTSMAN INTERNATIONAL LLC": "HUNTSMAN",
    "BREAKTHROUGH ENERGY ACTION": "BREAKTHROUGH ENERGY",
    "GEOTHERMAL EXCHANGE ORGANIZATION (GEO)": "GEOTHERMAL EXCHANGE ORGANIZATION",
    "ENVIRONMENTAL DEFENSE ACTION FUND, INC": "ENVIRONMENTAL DEFENSE FUND",
    "FGS GLOBAL (US) LLC ON BEHALF OF ENVIRONMENTAL DEFENSE ACTION FUND": "ENVIRONMENTAL DEFENSE FUND",
    "HERITAGE ACTION FOR AMERICA": "HERITAGE FOUNDATION",
    "AMERICAN EXPLORATION & PRODUCTION COUNCIL (AXPC)": "AMERICAN EXPLORATION PRODUCTION COUNCIL",
    "AMERICAN EXPLORATION & PRODUCTION COUNCIL": "AMERICAN EXPLORATION PRODUCTION COUNCIL",
    "AMERICAN EXPLORATION AND PRODUCTION COUNCIL (AXPC)": "AMERICAN EXPLORATION PRODUCTION COUNCIL",
    "AIR CONDITIONING HEATING & REFRIGERATION INSTITUTE (FKA AIR CONDITIONING AND R)": "AIRCONDITIONING HEATING REFRIGERATION INSTITUTE",
    "AIR CONDITIONING, HEATING AND REFRIGERATION INSTITUTE (AHRI)": "AIRCONDITIONING HEATING REFRIGERATION INSTITUTE",
    "AMERICANS FOR TAX REFORM": "AMERICANS FOR TAX REFORM",
    "CENTER FOR AMERICAN PROGRESS ACTION FUND": "CENTER FOR AMERICAN PROGRESS",
    "BIPARTISAN POLICY CENTER ACTION": "BIPARTISAN POLICY CENTER",
    "BIPARTISAN POLICY CENTER ADVOCACY NETWORK": "BIPARTISAN POLICY CENTER",
    "CAPITOL SOLUTIONS ON BEHALF OF BIPARTISAN POLICY CENTER ADVOCACY NETWORK, INC.": "BIPARTISAN POLICY CENTER",
    "BIPARTISAN POLICY CENTER ADVOCACY NETWORK(FOR KNWN AS BIPARTISAN ADVOCACY CENTER": "BIPARTISAN POLICY CENTER",
    "BIPARTISAN POLICY CENTER ADVOCACY NETWORK, INC.": "BIPARTISAN POLICY CENTER",
    "BIPARTISAN POLICY CENTER": "BIPARTISAN POLICY CENTER",
    "NATIONAL TAXPAYERS UNION": "NATIONAL TAXPAYERS UNION",
    "NATL ASSN OF TOWNS & TOWNSHIPS (NATAT)": "NATIONAL ASSOCIATION TOWNS TOWNSHIPS",
    "ALUTIIQ MANAGEMENT SERVICES": "ALUTIIQ",
    "INDIGO AGRICULTURE, INC.": "INDIGO",
    "AMERICAN FUEL & PETROCHEMICAL MANUFACTURERS (AFPM)": "AMERICAN FUEL PETROCHEMICAL MANUFACTURERS",
    "AMERICAN FUEL AND PETROCHEMICAL MANUFACTURERS (AFPM)": "AMERICAN FUEL PETROCHEMICAL MANUFACTURERS",
    "INDIGO AGRICULTURE": "INDIGO",
    "NATIONAL STONE SAND AND GRAVEL ASSOCIATION": "NATIONAL STONE SAND GRAVEL ASSOCIATION",
    "U S CHAMBER INSTITUTE FOR LEGAL REFORM": "US CHAMBER OF COMMERCE",
    "NESTLE USA": "NESTLE",
    "HAUCK STRATEGIES, LLC (OBO HOLLYFRONTIER CORPORATION)": "HOLLY FRONTIER",
    "NUTRIEN US LLC (FORMERLY KNOWN AS AGRIUM U.S. INC.) (A NUTRIEN LTD SUBSIDIARY)": "NUTRIEN",
    "NUTRIEN US LLC": "NUTRIEN",
    "HF SINCLAIR CORPORATION": "HOLLY FRONTIER",
    "VALERO ENERGY CORPORATION": "VALERO",
    "VALERO ENERGY": "VALERO",
    "AMERICAN COALITION FOR CLEAN COAL ELECTRICITY": "AMERICAS POWER",
    "AMERICAN COALITION FOR CLEAN COAL ELECTRICITY (ACCCE)": "AMERICAS POWER",
    "BOARDWALK PIPELINES, LLC": "BOARDWALK PIPELINE",
    "HYDROGEN ENERGY CALIFORNIA (HECA) (FORMERLY HYDROGEN ENERGY INTERNATIONAL LLC)": "HYDROGEN ENERGY CALIFORNIA",
    "VOLVO CAR CORPORATION": "VOLVO",
    "TELVENT USA LLC": "SCHNEIDER ELECTRIC",
    "EDPR OFFSHORE NORTH AMERICA LLC": "EDP RENEWABLES",
    "DOW CHEMICAL COMPANY DBA DOW": "DOW",
    "DOW CHEMICAL COMPANY": "DOW",
    "GEOTHERMAL EXCHANGE ORGANIZATION (GEO)": "GEOTHERMAL EXCHANGE ORGANIZATION",
    "TRUCKEE MEADOWS WATER AUTHORITY": "TRUCKEE MEADOWS WATER AUTHORITY",
    "THE DOW CHEMICAL COMPANY": "DOW",
    "THE ST. PETERSBURG DOWNTOWN PARTNERSHIP, INC.": "ST PETERSBURG DOWNTOWN PARTNERSHIP",
    "DOWNEY BRAND LLP (THREE RIVERS LEVEE IMPROVEMENT AUTHORITY)": "THREE RIVERS LEVEE IMPROVEMENT AUTHORITY",
    "WINDOW AND DOOR MANUFACTURERS ASSOCIATION": "WINDOW AND DOOR MANUFACTURERS ASSOCIATION",
    "WINDOW & DOOR MANUFACTURERS ASSOCIATION": "WINDOW DOOR MANUFACTURERS ASSOCIATION",
    "JELD-WEN WINDOWS & DOORS": "JELDWEN WINDOWS DOORS",
    "MYRTLE BEACH DOWNTOWN REDEVELOPMENT CORP": "MYRTLE BEACH DOWNTOWN REDEVELOPMENT",
    "DOW AGRO SCIENCES LLC": "DOW",
    "DOWNTOWN JACKSON PARTNERS INC": "DOWNTOWN JACKSON PARTNERS",
    "MOHR DAVIDOW VENTURES": "MOHR DAVIDOW VENTURES",
    "DOW CORNING CORPORATION": "DOW",
    "BILTBEST WINDOWS AND PATIO DOORS, INC.": "BILTBEST WINDOWS AND PATIO DOORS",
    "CITY OF DOWNEY CA": "DOWNEY CA",
    "VISA USA, INC.": "VISA",
    "TELEVISAUNIVISION, INC. (FKA UNIVISION COMMUNICATIONS INC.)": "UNIVISION",
    "CITY OF VISALIA, CA": "VISALIA CA",
    "UNITED STATES STEEL CORPORATION": "US STEEL",
    "SOLAR ENERGY INDUSTRIES ASSOCIATION (SEIA)": "SOLAR ENERGY INDUSTRIES ASSOCIATION",
    "RWE CLEAN ENERGY, LLC (FORMERLY RWE RENEWABLES AMERICAS, LLC)": "RWE",
    "RWE ENERGY": "RWE",
    "RWE OFFSHORE WIND SERVICES, LLC": "RWE",
    "RWE RENEWABLES AMERICAS, LLC": "RWE",
    "RWE RENEWABLES AMERICAS LLC (FORMERLY KNOWN AS E ON NORTH AMERICA)": "RWE",
    "EDF RENEWABLE ENERGY": "EDF RENEWABLES",
    "EDF RENEWABLES DEVELOPMENT, INC.": "EDF RENEWABLES",
    "GENERAL ATOMICS AERONAUTICAL SYSTEMS": "GENERAL ATOMICS",
    "GENERAL ATOMICS, ELECTROMAGNETIC SYSTEMS": "GENERAL ATOMICS",
    "USEC": "CENTRUS",
    "USEC INC": "CENTRUS",
    "CONSTELLATION ENERGY GENERATION, LLC, FORMERLY REPORTING AS EXELON CORPORATION": "CONSTELLATION ENERGY",
    "EXELON BUSINESS SERVICES LLC": "EXELON",
    "CONSTELLATION ENERGY GENERATION, LLC": "CONSTELLATION ENERGY",
    "PEPCO HOLDINGS INC": "EXELON",
    "CONSTELLATION ENERGY GROUP": "CONSTELLATION ENERGY",
    "EXELON BUSINESS SERVICES LLC (FORMERLY KNOWN AS CONSTELLATION ENERGY)": "EXELON",
    "EXELON BUSINESS SERVICES (FORMERLY CONSTELLATION ENERGY)": "EXELON",
    "CONSTELLATION ENERGY GROUP INC": "CONSTELLATION ENERGY",
    "CONSTELLATION ENERGY": "CONSTELLATION ENERGY",
    "EXELON BUSINESS SERVICES COMPANY": "EXELON",
    "AMERICAN EXPLORATION AND PRODUCTION COUNCIL (AXPC)": "AMERICAN EXPLORATION PRODUCTION COUNCIL",
    "ANADARKO PETROLEUM CORPORATION": "ANADARKO",
    "ANADARKO PETROLEUM CORPORATION (FORMERLY KERR MCGEE CORPORATION)": "ANADARKO",
    "MONROE COUNTY, FLORIDA": "MONROE COUNTY FL",
    "NATIONAL ALLIANCE OF FOREST OWNERS (NAFO)": "NATIONAL ALLIANCE OF FOREST OWNERS",
    "TOTAL CONTAINMENT": "TOTAL CONTAINMENT",
    "ENERGY TRANSFER EQUITY LP": "ENERGY TRANSFER",
    "ENERGY TRANSFER PARTNERS": "ENERGY TRANSFER",
    "TOTAL MILITARY MANAGEMENT": "TOTAL MILITARY MANAGEMENT",
    "TOTAL ADMINISTRATIVE SERVICES CORP. (TASC)": "TOTAL ADMINISTRATIVE SERVICES CORP TASC",
    "LITTLE SHELL TRIBE OF CHIPPEWA INDIANS": "LITTLE SHELL TRIBE OF CHIPPEWA INDIANS",
    "DUKE UNIVERSITY": "DUKE UNIVERSITY",
    "GOODYEAR TIRE & RUBBER COMPANY": "GOODYEAR",
    "THE GOODYEAR TIRE & RUBBER COMPANY": "GOODYEAR",
    "CITY OF GOODYEAR, AZ": "GOODYEAR AZ",
    "GOODYEAR TIRE AND RUBBER COMPANY": "GOODYEAR",
    "ALCALDE & FAY (CITY OF GOODYEAR, AZ)": "GOODYEAR AZ",
    "DELTA AIR LINES INC": "DELTA",
    "DELTA DENTAL PLANS ASSOCIATION": "DELTA DENTAL",
    "DELTA": "DELTA",
    "GOVERNMENT COUNSEL LLC ON BEHALF OF DELTA DENTAL OF CA": "DELTA DENTAL",
    "DELTA AIR LINES INC AND ITS SUBSIDIARY NORTHWEST AIRLINES INC (FKA DELTA AI": "DELTA",
    "DELTA AIR LINES, INC.": "DELTA",
    "DELTA AIR LINES": "DELTA",
    "DELTA DIABLO SANITATION DISTRICT": "DELTA DIABLO SANITATION DISTRICT",
    "NORTHWEST AIRLINES INC (ACQUIRED BY DELTA AIR LINES INC )": "DELTA",
    "DELTA AIR LINES INC AND ITS SUB NORTHWEST AIRLINES INC (FKA NORTHWEST AIRL": "DELTA",
    "DELTA STAR INC.": "DELTA STAR",
    "NATIONAL RURAL ELECTRIC COOPERATIVE ASSOCIATION (NRECA)": "NATIONAL RURAL ELECTRIC COOPERATIVE ASSOCIATION",
    "NEXTERA ENERGY INC": "NEXTERA",
    "EXXON MOBIL CORP": "EXXON",
    "CEMEX INC.": "CEMEX",
    "EVERGY FORMERLY KANSAS CITY POWER AND LIGHT COMPANY": "EVERGY",
    "USEC INC": "USEC",
    "USEC": "USEC",
    "PORTLAND GENERAL ELECTRIC CO": "PORTLAND GENERAL ELECTRIC",
    "PORTLAND GENERAL ELECTRIC": "PORTLAND GENERAL ELECTRIC",
    "BERKSHIRE HATHAWAY ENERGY": "BERKSHIRE HATHAWAY ENERGY",
    "SEMPRA": "SEMPRA",
    "COVESTRO LLC": "COVESTRO",
    "CMS ENERGY CORP": "CMS ENERGY",
    "DRAX GROUP": "DRAX GROUP",
    "SIEMENS ENERGY": "SIEMENS",
    "NIKOLA CORPORATION": "NIKOLA",
    "FUEL CELL AND HYDROGEN ENERGY ASSOCIATION": "FUEL CELL AND HYDROGEN ENERGY ASSOCIATION",
    "MARATHON PETROLEUM COMPANY LP": "MARATHON",
    "WEYERHAEUSER CO": "WEYERHAEUSER",
    "TOYOTA MOTOR NORTH AMERICA INC (TMA)": "TOYOTA",
    "UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL": "UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL",
    "AMERICAN BANKERS ASSOCIATION": "AMERICAN BANKERS ASSOCIATION",
    "UNIVERSITY OF IOWA": "UNIVERSITY OF IOWA",
    "FLORIDA CRYSTALS CORPORATION": "FLORIDA CRYSTALS",
    "CREDIT SUISSE SECURITIES (USA)": "CREDIT SUISSE",
    "ORANGE COUNTY TRANSPORTATION AUTHORITY": "ORANGE COUNTY TRANSPORTATION AUTHORITY",
    "AMERIQUAL GROUP LLC": "AMERIQUAL",
    "IRRIGATION ASSOCIATION": "IRRIGATION ASSOCIATION",
    "CEMEX INC.": "CEMEX",
    "GENERAL FEDERATION OF WOMEN'S CLUBS": "GENERAL FEDERATION OF WOMENS CLUBS",
    "RHEEM MANUFACTURING COMPANY": "RHEEM MANUFACTURING",
    "JPMORGAN CHASE & CO.": "JP MORGAN CHASE",
    "TESLA MOTORS, INC.": "TESLA",
    "UNIVERSITY OF ROCHESTER": "UNIVERSITY OF ROCHESTER",
    "AMERIPRISE FINANCIAL, INC.": "AMERIPRISE FINANCIAL",
    "EATON CORPORATION": "EATON",
    "BP AMERICA INC": "BP AMERICA",
    "HOLCIMUS FKA LAFARGEHOLCIM": "HOLCIM",
    "SOUTHWIRE CO": "SOUTHWIRE",
    "EL PASO CORPORATION": "EL PASO",
    "ALLIANCE TO SAVE ENERGY": "ALLIANCE TO SAVE ENERGY",
    "NATIONAL CORN GROWERS ASSOCIATION": "NATIONAL CORN GROWERS ASSOCIATION",
    "ALASKA RAILROAD CORP": "ALASKA RAILROAD",
    "CHAMBERS CONLON AND HARTWELL (MUNICIPALITY OF ANCHORAGE/PORT OF ANCHORAGE)": "CHAMBERS CONLON AND HARTWELL MUNICIPALITY OF ANCHORAGEPORT OF ANCHORAGE",
    "ALYESKA PIPELINE SERVICE COMPANY": "ALYESKA PIPELINE SERVICE",
    "ALUTIIQ MANAGEMENT SERVICES": "ALUTIIQ MANAGEMENT SERVICES",
    "BOEING COMPANY": "BOEING",
    "STATE OF NEW JERSEY": "STATE OF NEW JERSEY",
    "MISSISSIPPI MILITARY COMMITTEES COUNCIL": "MISSISSIPPI MILITARY COMMITTEES COUNCIL",
    "WESTERN GROWERS ASSOCIATION": "WESTERN GROWERS ASSOCIATION",
    "TRUSTEES OF COLUMBIA UNIVERSITY IN THE CITY OF NEW YORK": "COLUMBIA UNIVERSITY",
    "TRUSTEES OF THE UNIVERSITY OF PENNSYLVANIA": "UNIVERSITY OF PENNSYLVANIA",
    "BLUE DIAMOND GROWERS": "BLUE DIAMOND GROWERS",
    "CHAMBER OF COMMERCE OF THE U.S.A.": "US CHAMBER OF COMMERCE",
    "MANATEE COUNTY FLORIDA COUNTY ADMINISTRATOR'S OFFICE": "MANATEE COUNTY FLORIDA",
    "MECKLENBURG COUNTY": "MECKLENBURG COUNTY",
    "UNIVERSITY OF PITTSBURGH": "UNIVERSITY OF PITTSBURGH",
    "WAKE COUNTY NORTH CAROLINA": "WAKE COUNTY NORTH CAROLINA",
    "SCIENCE COALITION": "SCIENCE COALITION",
    "SOUTHERN RESEARCH INSTITUTE": "SOUTHERN RESEARCH INSTITUTE",
    "CAPITAL ONE FINANCIAL CORPORATION": "CAPITAL ONE FINANCIAL",
    "COALITION FOR AUTO REPAIR EQUALITY": "COALITION FOR AUTO REPAIR EQUALITY",
    "CITY OF BELLEVUE WA": "CITY OF BELLEVUE WA",
    "PORT OF HOOD RIVER OREGON": "PORT OF HOOD RIVER OREGON",
    "AMERICAN FOREST RESOURCE COUNCIL": "AMERICAN FOREST RESOURCE COUNCIL",
    "CITY OF BURIEN (WA)": "CITY OF BURIEN WA",
    "SHIPBUILDERS COUNCIL OF AMERICA": "SHIPBUILDERS COUNCIL OF AMERICA",
    "YAMHILL COUNTY OREGON": "YAMHILL COUNTY OREGON",
    "PORT OF CASCADE LOCKS (FORMERLY: CITY OF CASCADE LOCKS)": "PORT OF CASCADE LOCKS",
    "PORT OF WALLA WALLA": "PORT OF WALLA WALLA",
    "INTERNATIONAL UNION OF PAINTERS AND ALLIED TRADES": "INTERNATIONAL UNION OF PAINTERS AND ALLIED TRADES",
    "INTERNATIONAL BROTHERHOOD OF TEAMSTERS": "INTERNATIONAL BROTHERHOOD OF TEAMSTERS",
    "INTERNATIONAL BUSINESS MACHINES CORPORATION (IBM)": "IBM",
    "PLASTICS INDUSTRY ASSOCIATION, INC.": "PLASTICS INDUSTRY ASSOCIATION",
    "HOLLY FRONTIER CORPORATION (FKA FRONTIER OIL CORPORATION)": "HOLLY FRONTIER",
    "NEXTERA ENERGY": "NEXTERA",
    "ASSURANT INC": "ASSURANT",
    "LARGE PUBLIC POWER COUNCIL": "LARGE PUBLIC POWER COUNCIL",
    "FORD MOTOR COMPANY": "FORD MOTOR",
    "EXELON BUSINESS SERVICES CO., LLC": "EXELON",
    "GARY WILLIAMS ENERGY CORPORATION": "GARY WILLIAMS ENERGY",
    "APPLIED MATERIALS INC": "APPLIED MATERIALS",
    "ENVIRONMENTAL DEFENSE ACTION FUND": "ENVIRONMENTAL DEFENSE FUND",
    "CANYON SNOW CONSULTING ON BEHALF OF SVB FINANCIAL GROUP": "SVB FINANCIAL GROUP",
    "USEC INC": "USEC",
    "MICHIGAN FARM BUREAU": "MICHIGAN FARM BUREAU",
    "CATERPILLAR INC": "CATERPILLAR",
    "DELTA DENTAL PLANS ASSOCIATION": "DELTA DENTAL PLANS",
    "STATE UNIVERSITY SYSTEM OF FLORIDA": "STATE UNIVERSITY SYSTEM OF FLORIDA",
}
ccs_df["client_rename"] = [
    portland_replace_dict[x] if x in list(portland_replace_dict.keys()) else r
    for x, r in zip(ccs_df.client_name, ccs_df.client_rename)
]

In [320]:
ccs_df.loc[ccs_df.client_rename == "EXELON-CONSTELLATION-PEPCO"].client_name.unique()

array([], dtype=object)

In [ ]:
ccs_df.loc[ccs_df.client_rename == "USEC"].client_name.unique()

In [ ]:

for i, whichco in enumerate(ar):
    print(whichco)
    print(
        ccs_df.loc[
            ccs_df.client_rename
            == whichco
            # ].clean_client_general_description.unique()
        ].client_name.unique()
    )
    print("")

In [ ]:
ccs_df.loc[
    [
        "EUROPEAN FEDERATION FOR TRANSPORT AND ENVIRONMENT" in x
        for x in ccs_df.client_name
    ]
].clean_client_general_description.unique()

In [ ]:
ids = ccs_df.loc[ccs_df.client_rename == "VALERO ENERGY"]["client_rename"].index
ccs_df.loc[ids, "client_rename"] = "VALERO"

In [ ]:
dict_to_yaml(
    {
        substitute(x, use_basename=True): "SHELL"
        for x in [
            "SHELL USA, INC. (FKA SHELL OIL COMPANY)",
            "SHELL OIL COMPANY" "SHELL USA, INC.",
            "SHELL OIL",
            "BG NORTH AMERICA",
            "SHELL EXPLORATION & PRODUCTION COMPANY",
            "ACORN CONSULTING OBO SHELL USA INC.",
            "JOCELYN HONG AND ASSOCIATES (ON BEHALF OF THE CHILES GROUP & GULF SHELL INST.)",
        ]
    },
    "tmp.yml",
)

In [ ]:
ccs_df.loc[
    ccs_df.client_rename == "UNITED STEEL PAPER FORESTRY RUBBER MANUFACTURING ENERGY"
].client_name.unique()

In [ ]:
ccs_df.loc[ccs_df.client_name == "LITTLE SHELL TRIBE OF CHIPPEWA INDIANS"][
    "client_rename"
] = "LITTLE SHELL TRIBE OF CHIPPEWA INDIANS"

In [ ]:
ccs_df.loc[ccs_df.clean_client_general_description == "Biofuels company"].clean_client_general_description.unique()

In [ ]:
sector_descriptions_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sector_company_description_terms.yml"
)
sector_mapping = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
mapped_orgs = []
for v in sector_mapping.values():
    mapped_orgs = mapped_orgs + v

In [ ]:
for i, s in enumerate(list(sector_mapping.keys())):
    print(f"{i}. {s}")

In [ ]:
new_vals = list(rename_dict.values())
new_cos = sorted(list(set([x for x in new_vals if x not in mapped_orgs])))
for i, n in enumerate(new_cos):
    # if "oil" in n.lower():
    print(f"- {n}")

In [ ]:
search_term_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/search_term_list.yml"
)

search_terms = search_term_dict["search_term_list"]
probably_ccs = search_term_dict["probably_ccs"]
maybe_ccs = search_term_dict["maybe_ccs"]

terms = []
for t in search_terms:
    if "," in t:
        terms.append(t.replace('"', "").split(","))
    else:
        terms.append([t.replace('"', "")])

terms = [[substitute(t) for t in tt] for tt in terms]

single_terms = []
multiple_terms = []
for x in terms:
    if len(x) == 1:
        single_terms.append(x[0])
    else:
        multiple_terms.append(x)
print(single_terms)
print(multiple_terms)

In [ ]:
ccs_bills = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/ccs_laws.yml"
)["mostly_ccs_provisions"]
ccs_bills = [re.sub(r"[^\w\s]", "", x) for x in ccs_bills]
print(ccs_bills)

In [ ]:
from utils.io import dict_to_yaml

dict_to_yaml(
    {
        "telecom": list(
            df.loc[
                [
                    ("cable" in x)
                    | ("telecom" in x)
                    | ("television" in x)
                    | ("broadband" in x)
                    | ("broadcasting" in x)
                    | ("broadcasting" in n)
                    | ("phone" in x)
                    | ("cellular" in x)
                    | ("internet" in x)
                    | ("telecom" in n)
                    | ("television" in n)
                    | ("phone" in n)
                    | ("internet" in n)
                    | ("media" in x)
                    | ("media" in n)
                    | ("music" in x)
                    | ("music" in n)
                    | (" TV " in x)
                    | (" TV " in n)
                    for x, n in zip(
                        df.clean_client_general_description, df.client_rename
                    )
                ]
            ]
            .client_rename.value_counts()
            .index
        )
    },
    "tmp.yml",
)

In [ ]:
df = ccs_df.copy(deep=True)

In [ ]:
auto_transport_descriptions = [
    "auto",
    "car",
    "transport",
    "engine",
    "vehicle",
    "shipping",
    "cargo",
    "airline",
    "airplane",
]
ccs_company_descriptions = [
    "capture",
    "ccs",
    "ccus",
    "storage of co2",
    "sequestration",
    "greenhouse gas reduction",
]
iron_and_steel_descriptions = [
    "iron",
    "steel",
]
utility_descriptions = [
    "utility",
    "electricity",
    "power company",
    "power and light",
    "power light",
    "utilities",
    "electric",
    "power generation",
    "power provider",
    "energy provider",
    "energy generation",
    "energy production",
    "energy transmission",
    "energy distribution",
    "generation and distribution",
    "electric and natural gas",
    "transmission and distribution",
    "production and distribution",
    "transmission provider",
]
oil_and_gas_descriptions = [
    "oil and gas",
    "natural gas",
    "midstream",
    "drilling",
    "downstream",
    "upstream",
    "fuel",
    "pipeline",
    "refinery",
    "refining",
    "exploration and production",
    "exploration production",
    "E&P",
    "E & P",
    "oil",
    "petroleum",
]
df["transport_company"] = [
    terms_present(
        x,
        auto_transport_descriptions,
    )
    | (n in sector_mapping["auto and transportation"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["steel_company"] = [
    terms_present(
        x,
        iron_and_steel_descriptions,
    )
    | (n in sector_mapping["iron and steel"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["healthcare_company"] = [
    (
        terms_present(
            x,
            healthcare_company_descriptions,
        )
    )
    | (terms_present(n, healthcare_company_descriptions))
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["ccs_company"] = [
    terms_present(
        x,
        ccs_company_descriptions,
    )
    | (n in sector_mapping["ccs"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["utility"] = [
    terms_present(
        x,
        utility_descriptions,
    )
    | (n in sector_mapping["power generation and utilities"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["oilandgas"] = [
    terms_present(
        x,
        oil_and_gas_descriptions,
    )
    | (n in sector_mapping["oil and gas"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["probably ccs"] = [terms_present(x, probably_ccs) for x in df.clean_description]
df["maybe ccs"] = [
    terms_present(
        x,
        maybe_ccs,
    )
    for x in df.clean_description
]
df["health_care"] = [
    terms_present(x, ["healthcare", "health care", "medicine", "medical"])
    for x in df.clean_description
]
df["eor"] = [
    terms_present(x, ["EOR", "enhanced oil recovery"]) for x in df.clean_description
]
df["sustainable"] = [
    terms_present(x, ["sustainable", "sustainability"]) for x in df.clean_description
]
df["permitting"] = [
    terms_present(x, ["permit", "permitting", "permits"]) for x in df.clean_description
]
df["carbon tax"] = [
    terms_present(x, ["carbon tax", "tax on carbon"]) for x in df.clean_description
]
df["recycle"] = [
    terms_present(x, ["recycle", "recycling"]) for x in df.clean_description
]
df["hydrogen"] = [
    terms_present(
        x, ["hydrogen", "lowcarbon fuel", "low carbon fuel", "fuelcell", "fuel cell"]
    )
    for x in df.clean_description
]
df["nuclear"] = [terms_present(x, ["nuclear", "uranium"]) for x in df.clean_description]
df["carbon_management"] = [
    terms_present(x, ["carbon management", "carbonmanagement"])
    for x in df.clean_description
]
df["clean_energy"] = [terms_present(x, ["clean energy"]) for x in df.clean_description]
df["biofuels"] = [
    terms_present(x, ["ethanol", "bioethanol", "biofuel", "biodiesel"])
    for x in df.clean_description
]
df["climate"] = [terms_present(x, ["climate"]) for x in df.clean_description]
df["energy security"] = [
    terms_present(x, ["energy security"]) for x in df.clean_description
]

df["low_carbon"] = [
    terms_present(x, ["low carbon", "low emission"]) for x in df.clean_description
]
df["direct air"] = [
    terms_present(x, ["directair", "direct air", "air capture"])
    for x in df.clean_description
]
df["tax"] = [terms_present(x, ["tax"]) for x in df.clean_description]
df["carbon_tax"] = [
    terms_present(x, ["carbon tax", "carbontax", "tax on co2", "tax on carbon"])
    for x in df.clean_description
]
df["coal"] = [terms_present(x, ["coal"]) for x in df.clean_description]
df["clean coal"] = [
    terms_present(x, ["clean coal", "cleancoal"]) for x in df.clean_description
]
df["ccs_single"] = [terms_present(x, single_terms) for x in df.clean_description]
df["ccs_double"] = [
    any([terms_present(x, y, find_any=False) for y in multiple_terms])
    for x in df.clean_description
]
df["ccs_description"] = [
    max(sgl, dbl) for sgl, dbl in zip(df["ccs_single"], df["ccs_double"])
]

df["ccs_bills"] = [terms_present(x, ccs_bills) for x in df.clean_description]
df["definitely_ccs"] = [
    1 if (d + b) > 0 else 0 for d, b in zip(df.ccs_description, df.ccs_bills)
]
df["likely_ccs"] = [
    1 if (d + b + p + c) > 0 else 0
    for d, b, p, c in zip(
        df.ccs_description, df.ccs_bills, df["probably ccs"], df.ccs_company
    )
]
df["potentially ccs"] = [
    1 if (d + m + b + p + c) > 0 else 0
    for d, b, m, p, c in zip(
        df.ccs_description,
        df.ccs_bills,
        df["maybe ccs"],
        df["probably ccs"],
        df.ccs_company,
    )
]
df["net"] = [
    terms_present(
        x,
        [
            "negative emissions technology",
            "negative emissions technologies",
            "negative emission technology",
            "negative emission technologies",
        ],
    )
    for x in df.clean_description
]

In [ ]:
df[
    [
        "healthcare_company",
        "transport_company",
        "oilandgas",
        "utility",
        "steel_company",
        "ccs_company",
        "sustainable",
        "ccs",
        "probably ccs",
        "maybe ccs",
        "nuclear",
        "permitting",
        "low_carbon",
        "ccs_description",
        "definitely_ccs",
        "likely_ccs",
        "potentially ccs",
        "tax",
        "carbon_tax",
        "biofuels",
        "direct air",
        "clean coal",
        "clean_energy",
        "carbon_management",
        "hydrogen",
        "recycle",
        "carbon tax",
        "climate",
        "energy security",
        "ccs_bills",
        "health_care",
        "net",
        "eor",
    ]
].sum()

In [ ]:
print(
    len(
        df.loc[
            (df.definitely_ccs == 1)
            | (df.ccs_company == 1)
            | (df["probably ccs"] == 1)
            | ((df.oilandgas == 1) & (df["maybe ccs"] == 1))
            | ((df.utility == 1) & (df["maybe ccs"] == 1))
            | ((df.transport_company == 1) & (df["maybe ccs"] == 1))
            | ((df.steel_company == 1) & (df["maybe ccs"] == 1))
        ]
    )
)

df.loc[
    (df.ccs == 1)
    | (df.ccs_company == 1)
    | (df["probably ccs"] == 1)
    | ((df.oilandgas == 1) & (df["maybe ccs"] == 1))
    | ((df.utility == 1) & (df["maybe ccs"] == 1))
    | ((df.transport_company == 1) & (df["maybe ccs"] == 1))
    | ((df.steel_company == 1) & (df["maybe ccs"] == 1))
][["lobbyist_apportioned_usd", "activity_apportioned_usd"]].sum()

In [ ]:
for i, row in df.loc[
    (df["maybe ccs"] == 1)
    & ((df.oilandgas == 1) | (df.utility == 1))
    & (df.ccs_description == 0)
].iterrows():
    print(f"{row['client_rename']}: {row['clean_description']}")

In [ ]:
for i, row in df.loc[
    (df["probably ccs"] == 1) & (df.oilandgas == 0) & (df.ccs_description == 0)
].iterrows():
    print(f"{row['client_rename']}: {row['clean_description']}")

In [ ]:
# apportion total lobbying dollars spent (on filing) to individual lobbying activities, using two methods
df["activity_apportioned_usd"] = [
    usd / number_lobbying
    for usd, number_lobbying in zip(
        df.dollars_spent_lobbying, df.total_number_lobbying_activities
    )
]
df["lobbyist_apportioned_usd"] = [
    usd * (n_activity_lobbyists / total_lobbyists)
    if total_lobbyists > 0
    else activity_apportioned
    for usd, n_activity_lobbyists, total_lobbyists, activity_apportioned in zip(
        df.dollars_spent_lobbying,
        df.n_lobbyists_for_activity,
        df.total_number_of_lobbyists_on_filing,
        df.activity_apportioned_usd,
    )
]

In [ ]:
df.loc[df["potentially ccs"] == 1][
    ["lobbyist_apportioned_usd", "activity_apportioned_usd"]
].sum()

In [ ]:
sector_assignments = yaml_to_dict("sectors.yml")


all_companies = []
for _, value in sector_assignments.items():
    all_companies = all_companies + value
print(all_companies)

company_sector_dict = {}
for k, vv in sector_assignments.items():
    for v in vv:
        company_sector_dict = company_sector_dict | {v: k}

In [ ]:
company_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
lumped_sector_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/lumped_sectors.yml"
)["lumped_sector_assignments"]
df["sector"] = df["client_rename"].copy()
df["sector"] = [company_sector_dict[c] for c in df.sector]
df["lumped_sector"] = [lumped_sector_dict[c] for c in df.sector]

In [ ]:
definitely_ccs_by_year_sector_df = (
    df.loc[df["definitely_ccs"] == 1][
        ["filing_year", "lumped_sector", "lobbyist_apportioned_usd"]
    ]
    .groupby(["filing_year", "lumped_sector"])
    .sum()
    .sort_values(by=["filing_year", "lobbyist_apportioned_usd"], ascending=False)
    .reset_index()
)
all_by_year_sector_df = (
    df[["filing_year", "lumped_sector", "lobbyist_apportioned_usd"]]
    .groupby(["filing_year", "lumped_sector"])
    .sum()
    .sort_values(by=["filing_year", "lobbyist_apportioned_usd"], ascending=False)
    .reset_index()
)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import seaborn as sns

In [ ]:
hue_order = sorted(list(set(lumped_sector_dict.values())))

colors = [
    "tab:green",
    "tab:blue",
    "darkorange",
    "tab:purple",
    "magenta",
    "tab:red",
    "crimson",
    "dimgrey",
    "rebeccapurple",
    "teal",
    "darkblue",
]
color_palette = sns.color_palette(colors)
palette = color_palette
fig, ax = plt.subplots(2, 1, figsize=(16, 10))
sns.barplot(
    ax=ax[0],
    data=definitely_ccs_by_year_sector_df,
    x="filing_year",
    y="lobbyist_apportioned_usd",
    hue="lumped_sector",
    palette=color_palette,
    hue_order=hue_order,
)
ax[0].grid()
ax[0].legend(ncol=2)
ax[0].set_xlim([5, 24])

sns.barplot(
    ax=ax[1],
    data=all_by_year_sector_df,
    x="filing_year",
    y="lobbyist_apportioned_usd",
    hue="lumped_sector",
    palette=color_palette,
    hue_order=hue_order,
)
ax[1].grid()
ax[1].legend(ncol=2, loc="center")
ax[1].set_xlim([5, 24])

In [ ]:
for x in df.client_rename.loc[~df.client_rename.isin(all_companies)].unique():
    print(x.replace(",", ""))

In [ ]:
for i, row in df.clean_description.value_counts().items():
    print(i)

In [ ]:
df.client_general_description.fillna("", inplace=True)
df.loc[
    [
        ("ohio" in x.lower())  # | ("e&p" in x.lower())
        # for x in df.client_general_description
        for x in df.client_rename
    ]
].client_rename.unique()

In [ ]:
for i, row in df.client_rename.value_counts().items():
    print(f"{i}")

In [ ]:
df[["lobbyist_apportioned_usd", "client_rename"]].loc[df.ccs == 1].groupby(
    "client_rename"
).sum().sort_values(by="lobbyist_apportioned_usd", ascending=False).head(60)

In [ ]:
greenwashing_df.filing_id.nunique()

In [ ]:
greenwashing_df.loc[greenwashing_df.ccs == 1].groupby(
    "filing_id"
).first().reset_index().groupby("who_is_lobbying")["dollars_spent_lobbying"].sum()

In [ ]:
xom_entities = [x for x in df.columns if x in entities]
tmp = df[xom_entities].sum()
subset_entities = list(tmp[tmp > 0].index)
greenwashing_df = df.loc[df.ccs == 1].copy()
greenwashing_df[subset_entities].sum()

In [ ]:
search_string = '"carbon capture"OR"capture of carbon"OR"capture of co2"OR"CCUS"OR"CCS"OR"storage of carbon dioxide"OR"storage of co2"OR"capture and sequestration of carbon dioxide"OR"capture and sequestration of co2"OR"blue hydrogen"OR"hydrogen hub"OR"clean hydrogen"OR"45Q"OR"45V"OR"inflation reduction act"OR"117-169"OR"117-58"'

In [ ]:
query_filings = f"https://lda.senate.gov/api/v1/filings/?filing_specific_lobbying_issues={search_string}&filing_period=year_end"

f = requests.get(query_filings)

### Info about congress.gov search for bills with ccs relevance

In [ ]:
#query for congress.gov search
https://www.congress.gov/u/ISk7JcnkfFMeJIdw4sMwB

# "carbon capture" OR "capture of carbon" OR "CCUS" OR "CCS" OR "storage of carbon dioxide" OR "storage of co2" OR "sequestration of carbon dioxide" OR "sequestration of co2" OR "blue hydrogen" OR "clean hydrogen" OR "45Q" OR "45V"

In [ ]:
xom_word_cloud = [
    x.replace("Discussions related to", "")
    .replace("provisions", "")
    .replace("Provisions", "")
    for x in df.description.to_list()
]
xom_word_cloud = [
    x.replace("related", "")
    .replace("Related", "")
    .replace("issues", "")
    .replace("provisions related", "")
    for x in xom_word_cloud
]

In [ ]:
from wordcloud import WordCloud

word_cloud = WordCloud(background_color="white")
word_cloud.generate(" ".join(xom_word_cloud))
import os
from PIL import Image
import numpy as np
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
df[
    [
        "lobbyist_registrant_name",
        "client_name",
        "who_is_lobbying",
        "general_issue_code",
        "description",
    ]
]

In [ ]:
query_congress = f"https://api.congress.gov/v3/bill/118/hr/1262/summaries?format=json&api_key={data_gov_api_key}"
s = requests.get(query_congress)
s.json()["summaries"][-1]["text"]

In [ ]:
def parse_dollars_spent(income, expense):
    if (income is None) & (expense is None):
        return "income and expense are none", 0.0
    if income is None:
        return "corporation lobbying for itself", float(expense)
    if expense is None:
        return "hired lobbying firm", float(income)
    else:
        return "both income and expense > $0", float(income) + float(expense)


def initialize_row(entity_df, result, filing_id):
    # set up row dictionary using entity booleans
    initialize_row_dict = dict(
        zip(
            [x.lower() for x in list(entity_df["name"])],
            [0] * len(list(entity_df["name"])),
        )
    )
    (
        initialize_row_dict["who_is_lobbying"],
        initialize_row_dict["dollars_spent_lobbying"],
    ) = parse_dollars_spent(result["income"], result["expenses"])
    initialize_row_dict["filing_id"] = filing_id
    initialize_row_dict["url"] = result["url"]
    initialize_row_dict["filing_year"] = int(result["filing_year"])
    initialize_row_dict["filing_period"] = result["filing_period"]
    initialize_row_dict["lobbyist_posted_by_name"] = result["posted_by_name"]

    initialize_row_dict["lobbyist_registrant_id"] = result["registrant"]["id"]
    initialize_row_dict["lobbyist_registrant_name"] = result["registrant"]["name"]
    initialize_row_dict["lobbyist_registrant_contact"] = result["registrant"][
        "contact_name"
    ]
    initialize_row_dict["client_id"] = result["client"]["id"]
    initialize_row_dict["client_client_id"] = result["client"]["client_id"]
    initialize_row_dict["client_name"] = result["client"]["name"]
    initialize_row_dict["affiliated_organizations_present"] = False
    if len(result["affiliated_organizations"]) > 0:
        initialize_row_dict["affiliated_organizations_present"] = True
    initialize_row_dict["convictions_present"] = False
    if len(result["conviction_disclosures"]) > 0:
        initialize_row_dict["convictions_present"] = True
    return initialize_row_dict

In [ ]:
def parse_lobbyists(df, lobbyists, details):
    lobby_dict = {}
    lobby_dict["firm_name"] = details["lobbyist_registrant_name"]
    lobby_dict["client_name"] = details["client_name"]
    lobby_dict["general_issue_code"] = details["general_issue_code"]
    lobby_dict["description"] = details["description"]
    lobby_dict["filing_period"] = details["filing_period"]
    lobby_dict["filing_year"] = details["filing_year"]
    lobby_dict["url"] = details["url"]
    lobby_dict["filing_id"] = details["filing_id"]

    lobby_list = []
    # unpack lobbyists list
    for lobbyist in lobbyists:
        lobby_dict["name"] = (
            lobbyist["lobbyist"]["last_name"]
            + ", "
            + lobbyist["lobbyist"]["first_name"]
        )
        lobby_dict["covered_position"] = "None"
        if "covered_position" in lobbyist:
            lobby_dict["covered_position"] = lobbyist["covered_position"]
        lobby_dict["id"] = lobbyist["lobbyist"]["id"]
        lobby_list.append(lobby_dict.copy())

    if df is None:
        df = pd.DataFrame(
            columns=[
                "name",
                "firm_name",
                "client_name",
                "general_issue_code",
                "description",
                "covered_position",
                "filing_year",
                "filing_period",
                "url",
            ]
        )

    df = pd.concat([df, pd.DataFrame(lobby_list)])
    return df

In [ ]:
# get govt entity names
entity_df = pd.DataFrame(government_entities)

corporations = [
    "Exxon",
    "ExxonMobil",
]  # 'exxon' alone includes ExxonMobil Exxon Mobil Corp, Exxon Mobil Corporation, etc. ['Exxon Mobil'.upper(),'ExxonMobil'.upper(),'Exxon'

row_list = (
    []
)  # initialize holder for each row (which corresponds to a single lobbying activity)
max_page = 20

filing_id = 0
lobbyists_df = None
for corporation in corporations:
    for year in list(range(1999, 2025)):
        page = 1
        while page < max_page:
            # print(f"Querying page {page} for year {year}")
            query_filings = f'https://lda.senate.gov/api/v1/filings/?client_name="{corporation}"&filing_period=year_end&filing_year={year}&page={page}'

            f = requests.get(query_filings)

            if "detail" in f.json():
                break

            results = f.json()["results"]
            page += 1  # increase the page for the next query
            # print(f"{len(results)} filings found")
            if len(results) < 25:
                page = max_page

            for result in results:
                row_dict_base = initialize_row(entity_df, result, filing_id)
                activities = result["lobbying_activities"]

                for activity_count, activity in enumerate(activities):
                    row_dict = row_dict_base.copy()
                    # set up row dictionary using entity booleans
                    row_dict["activity_count"] = activity_count
                    row_dict["general_issue_code"] = activity["general_issue_code"]
                    row_dict["description"] = activity["description"]
                    lobbyists_df = parse_lobbyists(
                        lobbyists_df, activity["lobbyists"], row_dict
                    )

                    lobbyist_id_list = []
                    for lobbyist in activity["lobbyists"]:
                        lobbyist_id_list.append(lobbyist["lobbyist"]["id"])

                    row_dict["lobbyist_ids"] = "; ".join(
                        ["None" if x is None else str(x) for x in lobbyist_id_list]
                    )
                    for entity in activity["government_entities"]:
                        row_dict[entity["name"].lower()] = 1

                    row_list.append(row_dict.copy())

                    row_dict.clear()
                filing_id += 1

tmp = pd.DataFrame(row_list)
entities_influenced = tmp[[x.lower() for x in list(entity_df["name"])]].sum()
zeroed = list(entities_influenced[entities_influenced == 0].index)
xom_df = tmp[[x for x in tmp.columns.values if x not in zeroed]]
xom_df = xom_df.drop_duplicates(subset=xom_df.columns.difference(["filing_id"]))
xom_unique_filing_ids = list(
    xom_df.filing_id.unique()
)  # keep a list of the non-duplicate filing ids

# keep only right company aliases
company_aliases = [
    "EXXON MOBIL CORP",
    "EXXONMOBIL CHEMICAL CO",
    "EXXON USA",
    "EXXON CO USA",
    "EXXON MOBIL CORPORATION",
    "EXXON MOBIL PRODUCTION CO",
    "EXXON MOBIL",
    "EXXONMOBIL CORPORATION",
    "EXXONMOBIL CORP",
    "EXXON MOBILE",
    "EXXONMOBIL GLOBAL SERVICES COMPANY",
    "DCI GROUP, LLC, ON BEHALF OF EXXONMOBIL",
    "EXXONMOBIL",
    "HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WILLIAMS LLP BEHALF OF EXXON MOBIL)",
    "EXXON MOBIL COPORATION",
    "EXXON MOBIL CORP.",
]
xom_df = xom_df.loc[xom_df.client_name.isin(company_aliases)]


lobbyists_df = lobbyists_df.loc[
    (lobbyists_df.client_name.isin(company_aliases))
    & (lobbyists_df.filing_id.isin(xom_unique_filing_ids))
]

xom_df["which_congress"] = [which_congress(y) for y in xom_df["filing_year"]]

In [ ]:
test_df[["which_congress", "description"]].loc[
    [x != "" for x in test_df.description]
].to_clipboard()

In [ ]:
# how much money has exxonmobil spent lobbying congress in 25 years?
xom_df[["dollars_spent_lobbying", "filing_id"]].groupby("filing_id").first()[
    "dollars_spent_lobbying"
].sum()

In [ ]:
xom_df.to_csv("xom_lda_filings.csv")
lobbyists_df.to_csv("xom_lobbyists.csv")

In [ ]:
tmp = (
    xom_df[
        [
            "dollars_spent_lobbying",
            "lobbyist_registrant_name",
            "filing_id",
            "filing_year",
        ]
    ]
    .groupby(["lobbyist_registrant_name", "filing_id"])
    .first()
    .reset_index()
)

In [ ]:
tmp.info()

In [ ]:
total_dollars_spent_df = (
    tmp.groupby("lobbyist_registrant_name")
    .agg(
        {
            "dollars_spent_lobbying": "sum",
            "filing_id": "count",
            "filing_year": "min",
            "filing_year": "min",
        }
    )
    .sort_values(by="dollars_spent_lobbying", ascending=False)
)
total_dollars_spent_df.columns = [
    "dollars_spent_lobbying",
    "total_lda_filings",
    "earliest_filing_year",
]
max_year_df = (
    tmp[["lobbyist_registrant_name", "filing_year"]]
    .groupby("lobbyist_registrant_name")
    .agg({"filing_year": "max"})
)
max_year_df.columns = ["latest_filing_year"]
total_dollars_spent_df = total_dollars_spent_df.merge(
    max_year_df, right_index=True, left_index=True
)
total_dollars_spent_df

In [ ]:
xom_dci_lobbyists = list(
    lobbyists_df.loc[lobbyists_df.firm_name == "DCI GROUP, L.L.C."].name.unique()
)

In [ ]:
counts_df = (
    lobbyists_df[["name", "firm_name"]]
    .groupby("firm_name")
    .count()
    .sort_values(by="name")
)
counts_df.columns = ["firm_total_filings"]
df = lobbyists_df.merge(counts_df, left_on="firm_name", right_index=True)
counts_df = (
    lobbyists_df[["name", "firm_name"]]
    .groupby("name")
    .count()
    .sort_values(by="firm_name")
)
counts_df.columns = ["total_number_of_filings"]
df = df.merge(counts_df, left_on="name", right_index=True)

In [ ]:
lobbyists_df[["name", "firm_name", "client_name", "total_number_of_filings"]].groupby(
    "name"
).first().sort_values(by="total_number_of_filings", ascending=False).head(50)

In [ ]:
lob1 = requests.get(
    'https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=1'
)
lob2 = requests.get(
    'https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=2'
)
dci_lobbyists = lob1.json()["results"] + lob2.json()["results"]

In [ ]:
all_dci_lobbyists = []
for dci in dci_lobbyists:
    all_dci_lobbyists.append(dci["last_name"] + ", " + dci["first_name"])

sorted(all_dci_lobbyists)

In [ ]:
xom_df.loc[xom_df.f]

In [ ]:
xom_dci_lobbyists

In [ ]:
xom_dict={}
for x in xom:
    xom_dict['year']=int(x['filing_year'])
    xom_dict['rest']=

In [ ]:
lob = requests.get("https://lda.senate.gov/api/v1/lobbyists/")
filing_types = requests.get(
    "https://lda.senate.gov/api/v1/constants/filing/filingtypes/"
)
general_issues = requests.get(
    "https://lda.senate.gov/api/v1/constants/filing/lobbyingactivityissues/"
)
govt_entities = requests.get(
    "https://lda.senate.gov/api/v1/constants/filing/governmententities/"
)
countries = requests.get("https://lda.senate.gov/api/v1/constants/general/countries/")
states = requests.get("https://lda.senate.gov/api/v1/constants/general/states/")
prefixes = requests.get("https://lda.senate.gov/api/v1/constants/lobbyist/prefixes/")
suffixes = requests.get("https://lda.senate.gov/api/v1/constants/lobbyist/suffixes/")
contribution_items = requests.get(
    "https://lda.senate.gov/api/v1/constants/contribution/itemtypes/"
)

In [ ]:
lob = requests.get(
    'https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=2'
)
lob.json()["results"]

In [ ]:
contributions = []
for page in range(1, 4):
    c = requests.get(
        f'https://lda.senate.gov/api/v1/contributions/?registrant_name="DCI GROUP, L.L.C."&filing_type=YY&page={page}'
    )
    contributions = contributions + c.json()["results"]

In [ ]:
row_list = []

for dci in contributions:
    row_dict = {}
    for i in dci["contribution_items"]:
        row_dict["contribution_type"] = i["contribution_type"]
        row_dict["contributor_name"] = i["contributor_name"]
        row_dict["payee_name"] = i["payee_name"]
        row_dict["honoree_name"] = i["honoree_name"]
        row_dict["usd"] = float(i["amount"])
        row_dict["date"] = i["date"]
        row_dict["filing_year"] = dci["filing_year"]
        row_dict["registrant_name"] = dci["registrant"]["name"]
        row_list.append(row_dict.copy())
dci_contributions_df = pd.DataFrame(row_list)

In [ ]:
def strip_terms(x, terms):
    for term in terms:
        x = x.replace(term, "")

    return x.split(" (")[0].title()

In [ ]:
terms = [
    "Senator ",
    "US ",
    "Rep. ",
    "U.S. ",
    "Representative ",
    "Sen. ",
    "Congressional ",
    "Candidate ",
    "Congressman ",
    "for ",
    "Friends of ",
]

print(dci_contributions_df.honoree_name.nunique())
print(dci_contributions_df.stripped_honoree_name.nunique())

In [ ]:
dci_contributions_df.honoree_name.unique()
terms = [
    "Senator ",
    "US ",
    "Rep. ",
    "Congresswoman ",
    "U.S. ",
    " 08",
    " Jr.",
    "Jr",
    "Resident Commissioner of Puerto Rico ",
    "Senate ",
    "Campaign",
    "Presidential ",
    "candidate ",
    "Candidate ",
    "Representative ",
    "Sen. ",
    "House ",
    "Congressional ",
    "Candidate ",
    "Representative-Elect ",
    "Campaign Committee",
    "Committee",
    "Congressman ",
    "for ",
    "Friends of ",
    " Senate",
    " Congress",
    "Friend of ",
    "- In-Kind Donation",
]
dci_contributions_df["stripped_honoree_name"] = [
    strip_terms(x, terms) for x in dci_contributions_df.honoree_name
]
dci_contributions_df.stripped_honoree_name.unique()

In [ ]:
yy = list(dci_contributions_df.stripped_honoree_name.unique())
dci_contributions_df["stripped_honoree_name"] = [
    x.strip() for x in dci_contributions_df["stripped_honoree_name"]
]
dci_contributions_df["stripped_honoree_name_sim"] = dci_contributions_df[
    "stripped_honoree_name"
]
for y in yy:
    dci_contributions_df["stripped_honoree_name_sim"] = [
        y if x in y else x for x in dci_contributions_df.stripped_honoree_name_sim
    ]
yy = list(dci_contributions_df.stripped_honoree_name_sim.unique())
for y in yy:
    dci_contributions_df["stripped_honoree_name_sim"] = [
        y if x in y else x for x in dci_contributions_df.stripped_honoree_name_sim
    ]

In [ ]:
yy = list(dci_contributions_df.stripped_honoree_name_sim.unique())
for term, replacement in {
    "Allan": "Alan",
    "Jim": "James",
    "Gordan": "Gordon",
    "&": "And",
    "Georgians": "Johnny",
    "M.": "Mary",
    "Pete": "Peter",
}.items():
    dci_contributions_df["stripped_honoree_name_sim"] = [
        x.replace(term, replacement)
        for x in dci_contributions_df.stripped_honoree_name_sim
    ]

In [ ]:
dci_contributions_df[["stripped_honoree_name_sim", "usd"]].groupby(
    "stripped_honoree_name_sim"
).sum().sort_values(by="usd", ascending=False).sum()

In [ ]:
import pandas as pd
from name_matching.name_matcher import NameMatcher

# define a dataset with bank names
df_a = pd.DataFrame({"recipient1": y})

# alter each of the bank names a bit to test the matching
df_b = pd.DataFrame({"recipient2": sorted(y)})

# initialise the name matcher
matcher = NameMatcher(
    number_of_matches=5, legal_suffixes=True, common_words=False, top_n=50, verbose=True
)

# adjust the distance metrics to use
matcher.set_distance_metrics(["bag", "typo", "refined_soundex"])

# load the data to which the names should be matched
matcher.load_and_process_master_data(
    column="recipient1", df_matching_data=df_a, transform=True
)

# perform the name matching on the data you want matched
matches = matcher.match_names(to_be_matched=df_b, column_matching="recipient2")

# combine the datasets based on the matches
# combined = pd.merge(df_a, matches, how='left', left_index=True, right_on='match_index')
# combined#combined = pd.merge(combined, df_b, how='left', left_index=True, right_index=True)

In [ ]:
# matches.drop(['match_name_0','score_0'],axis=1,inplace=True)
matches

In [ ]:
dci_contributions_df[["filing_year", "usd"]].groupby("filing_year").sum()

In [ ]:
firms = []
# lobbyists=lobbyists.json()['results']
for lobby in lobbyists:
    print(lobby)
    firms.append(lobby["registrant"]["name"])

In [ ]:
firms

In [ ]:
payee = "Friends of Max Baucus".replace(" ", "%20")
year = "2008"
contributor = "Crossroads Strategies, LLC".replace(" ", "%20")
year = "2023"
query_all_contributions = f"https://lda.senate.gov/api/v1/contributions/?contribution_payee={payee}&filing_year={year}"
c = requests.get(query_all_contributions)
baucus = c.json()["results"]

In [ ]:
contributor = "Crossroads Strategies".replace(" ", "%20")
year = "2023"
query_all_contributions = f"https://lda.senate.gov/api/v1/contributions/?contribution_contributor={contributor}&filing_year={year}"
c = requests.get(query_all_contributions)
crossroads = c.json()["results"]

In [ ]:
crossroads

In [ ]:
total = 0

for cc in baucus:
    contributions = cc["contribution_items"]
    for i in contributions:
        if i["payee_name"].lower() == "friends of max baucus":
            print(f'{i["contributor_name"]}, amount: ${i["amount"]}')
            total = total + float(i["amount"])

In [ ]:
total

In [ ]:
govt_entities_df = pd.DataFrame(govt_entities.json())
contribution_items_df = pd.DataFrame(contribution_items.json())
general_issues_df = pd.DataFrame(general_issues.json())
filing_types_df = pd.DataFrame(filing_types.json())

In [ ]:
constants = requests.get("https://lda.senate.gov/api/v1/constants/")
constants.json()

In [ ]:
r.json()

In [ ]:
rr = requests.get(r.json()["filings"] + "?client_name=exxon%20mobil&filing_year=2023")

In [ ]:
rr.json()

In [ ]:
rr.json()["results"][0].keys()

dictionary = rr.json()["results"][0]
dictionary = flatten(dictionary)

for key, value in dictionary.items():
    print(f"{key}: {value}")

In [ ]:
flatten(dictionary)

In [ ]:
query = f"https://developer.nrel.gov/api/pvwatts/v8.json?api_key={config['apikey']}&lat={lat}&lon={lon}&system_capacity={config['kw']}&azimuth={azimuth}&tilt={tilt}&array_type=1&module_type=1&losses=10"
                response = requests.get(query, timeout=20)
                info_dict["ac_annual"] = response.json()["outputs"]["ac_annual"]